In [ ]:
import sys
import os
from google.colab import drive

drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/rag/models'

sys.path.append(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain_community
!pip install rank_bm25
!pip install faiss-cpu
!pip install langchain_core
!pip install langchain_openai
!pip install chromadb
!pip install langchain_chroma
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_core
!pip install langchain_classic

In [ ]:
from datasets import Dataset
from google.colab import userdata

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-2012395113.py:11: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-2012395113.py:11: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics

In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

llm = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.01,
    max_tokens = 2000,
    openai_api_key = userdata.get('OPENAI_API_KEY')
)

db_path = "/content/drive/MyDrive/rag/chromadb_backup"
client = chromadb.PersistentClient(path=db_path)
collection_name = "tax_law"

try:
  collection = client.get_collection(name=collection_name)
except Exception:
  raise RuntimeError(f"Collection {collection_name} not found")

embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key = userdata.get("OPENAI_API_KEY"))

vectorstore = Chroma(client = client, collection_name=collection_name, embedding_function=embeddings)


## 답변 생성 확인

## version3

In [ ]:
from V3_ensemble import Pipeline, Reference

In [ ]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = Pipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당하면 결손금 소급공제 환급신청이 가능하며, 직전 사업연도에 중소기업이 아니더라도 환급이 인정됩니다 [[0]] [[3]].

[근거 법령 및 본문]법인세법 72조 ①항: 중소기업에 해당하는 내국법인은 각 사업연도에 결손금이 발생한 경우 대통령령으로 정하는 직전 사업연도의 법인세액(이하 이 조에서 "직전 사업연도의 법인세액"이라 한다)을 한도로 제1호의 금액에서 제2호의 금액을 차감한 금액을 환급 신청할 수 있다. <개정 2018.12.24>
법인세법 시행령 10조 ③항 3호: 법 제72조제1항 및 「조세특례제한법」 제8조의4에 따라 공제받은 결손금


## version2

In [ ]:
from V2_multiquery import MultiQueryPipeline, Reference

In [ ]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = MultiQueryPipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당하는 경우에는 결손금 소급공제 환급신청이 가능하며, 직전 사업연도에 중소기업이 아니었더라도 환급이 가능합니다[[0]].

[근거 법령 및 본문]소득세법 85조 ①항: 「조세특례제한법」 제6조제1항에 따른 중소기업을 경영하는 거주자가 그 중소기업의 사업소득금액을 계산할 때 제45조제3항에서 규정하는 해당 과세기간의 이월결손금(부동산임대업에서 발생한 이월결손금은 제외한다. 이하 이 조에서 같다)이 발생한 경우에는 직전 과세기간의 그 중소기업의 사업소득에 부과된 종합소득 결정세액을 한도로 하여 대통령령으로 정하는 바에 따라 계산한 금액(이하 "결손금 소급공제세액"이라 한다)을 환급신청할 수 있다. 이 경우 소급공제한 이월결손금에 대해서 제45조제3항을 적용할 때에는 그 이월결손금을 공제받은 금액으로 ...
법인세법 시행령 10조 ②항: 법 제13조제1항제1호에 따라 결손금을 공제할 때에는 먼저 발생한 사업연도의 결손금부터 차례대로 공제한다. <개정 2016.2.12, 2019.2.12>


## version1

In [ ]:
from V1_parent import ParentPipeline, Reference

In [ ]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = ParentPipeline(query, vectorstore, llm)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당되면 결손금 소급공제 환급신청을 통해 직전 사업연도에 납부한 법인세를 환급받을 수 있습니다.[[3]]

[근거 법령 및 본문]부가가치세법 시행령 91조 ①항 1호: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업 또는 같은 영 제6조의4제1항에 따른 중견기업에 해당하는 법인(「조세특례제한법」 제6조제3항제2호에 따른 제조업을 주된 사업으로 경영하는 기업에 한정한다)일 것


## version0

In [ ]:
from V0_basic import BasicPipeline, Reference

In [ ]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = BasicPipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당되면 결손금 소급공제 환급신청이 가능하며, 이는 직전 사업연도에 중소기업이 아니었더라도 결손금 발생 시점에 중소기업인 경우에 한정하여 환급이 허용되기 때문입니다 [[0], [3]].

[근거 법령 및 본문]부가가치세법 시행령 91조 ①항 2호 가목: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업인 경우: 직전 사업연도에 공급한 재화 또는 용역의 공급가액의 합계액에서 수출액이 차지하는 비율이 30퍼센트 이상이거나 수출액이 50억원 이상일 것
부가가치세법 시행령 91조 ①항 1호: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업 또는 같은 영 제6조의4제1항에 따른 중견기업에 해당하는 법인(「조세특례제한법」 제6조제3항제2호에 따른 제조업을 주된 사업으로 경영하는 기업에 한정한다)일 것


## ragas 기반 모델별 RAG 성능 평가

In [ ]:
from V4_ import Pipeline4
from V3_ensemble import Pipeline
from V2_multiquery import MultiQueryPipeline
from V1_parent import ParentPipeline
from V0_basic import BasicPipeline

In [ ]:
import nest_asyncio
import logging
from datasets import Dataset
from google.colab import userdata

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd
import json
from ragas.run_config import RunConfig

nest_asyncio.apply()

logging.getLogger("ragas").setLevel(logging.ERROR)
logging.getLogger("openai").setLevel(logging.ERROR)

def evaluation(version:str, n_tries: int = 3):
  file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json']
  category = ['법인세법', 'vat', '소득세', '양도소득세']

  final_summary = []
  best_perf = []

  llm_ragas = LangchainLLMWrapper(llm)
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

  run_config = RunConfig(max_retries=3, timeout=180, max_workers=1)

  for path, law_name in zip(file_path, category):

    try:
      with open(path, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

      try_results = []

      for i in range(n_tries):
        print(f"{law_name}: {i+1}번째 evaluation")

        answers = []
        contexts = []
        questions = []
        ground_truths = []

        for item in test_data:
          query = item['query']
          try:
            if version == 'v4':
              pipeline = Pipeline4(query, llm, vectorstore)
            elif version == 'v3':
              pipeline = Pipeline(query, llm, vectorstore)
            elif version == 'v2':
              pipeline = MultiQueryPipeline(query, llm, vectorstore)
            elif version == 'v1':
              pipeline = ParentPipeline(query, vectorstore, llm)
            elif version == 'v0':
              pipeline = BasicPipeline(query, llm, vectorstore)
            else:
              raise ValueError("v0, v1, v2, v3, v4 중 선택하세요.")

            answer, _, retrieved_docs = pipeline.generate_answer()
            answers.append(answer)
            contexts.append([doc.page_content for doc in retrieved_docs])
            questions.append(query)
            ground_truths.append(item['answer'])

          except Exception as e:
            print(f"평가 중 에러 발생: {e}")
            continue

        data = {
          "user_input": questions,
          "response": answers,
          "retrieved_contexts": contexts,
          "reference": ground_truths
        }

        dataset = Dataset.from_dict(data)

        result = evaluate(
            dataset=dataset,
            metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
            llm=llm_ragas,
            embeddings=embedding_ragas,
            run_config=run_config
        )

        df_try = result.to_pandas()
        df_try['try_no'] = i + 1
        try_results.append(df_try)

      if try_results:
        all_tries_df = pd.concat(try_results, ignore_index=True)
        cols = ['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']

        avg = all_tries_df[cols].mean()
        print(f"{law_name} {n_tries}회 평균")
        print(avg)

        all_tries_df['sum'] = all_tries_df[cols].sum(axis=1)
        try_scores = all_tries_df.groupby('try_no')['sum'].mean()
        best_try_idx = int(try_scores.idxmax())
        best_try = all_tries_df[all_tries_df['try_no'].values == best_try_idx].copy()
        best_try['law_name'] = law_name

        print(f"best performance: {best_try_idx}번째")
        display(best_try)
        best_perf.append(best_try)

        summary = avg.to_frame().T
        summary['law_name'] = law_name
        final_summary.append(summary)

    except Exception as e:
      print(f"{law_name} 평가 중 오류 발생: {e}")


  if final_summary:
      final_report = pd.concat(final_summary, ignore_index=True)
      print("-"*50)
      print("최종 평가 결과 요약")
      display(final_report.set_index('law_name'))
      return final_report

  return None


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-2248849936.py:13: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-2248849936.py:13: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics

In [ ]:
eval0 = evaluation('v0')

/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.453333
answer_relevancy     0.579804
context_recall       1.000000
context_precision    0.750791
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,"공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우, 양도세 ...","현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.000000,0.549197,1.0,0.383333,2,2.932531,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 제83조의 규정에 따라 1세...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.000000,0.971684,1.0,1.000000,2,2.971684,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도하는 경우, 제55조제1항에 따른 세율에 20% 또는 30%...",1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,0.666667,0.734813,1.0,0.948413,2,3.349893,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","증여자가 사망한 후 배우자가 양도하는 경우, 비과세 여부는 제공된 자료에서 명확히 ...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.666667,0.000000,1.0,0.854167,2,2.520833,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,국민주택채권 등 매각차손은 필요경비에 산입되지 않습니다.,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,0.000000,0.836059,1.0,0.596190,2,2.432249,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.555556,0.588001,0.977778,0.825057
vat,0.441667,0.577041,1.000000,0.676745
소득세,0.538889,0.531130,0.732937,0.624016
양도소득세,0.453333,0.579804,1.000000,0.750791


In [ ]:
eval1 = evaluation('v1')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.227778
answer_relevancy     0.601601
context_recall       1.000000
context_precision    0.677477
dtype: float64
best performance: 3번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
10,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에는 양도세...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",0.50,0.911726,1.0,0.327778,3,2.739504,양도소득세
11,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 세법상 가족으로서 배우자와 ...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.00,0.971698,1.0,1.000000,3,2.971698,양도소득세
12,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도한 경우, 각각의 양도소득에 대해 별도로 양도소득세가 과세되...",1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,0.00,0.765752,1.0,0.842679,3,2.608432,양도소득세
13,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","배우자가 사망 후 양도하는 경우, 해당 양도는 증여로 간주되지 않으므로 비과세 여부...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.25,0.000000,1.0,0.680158,3,1.930158,양도소득세
14,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,제공된 자료에서 국민주택채권 등 매각차손 필요경비 산입 여부에 관한 구체적인 내용은...,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,1.00,0.000000,1.0,0.620297,3,2.620297,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.433333,0.568665,0.950000,0.787904
vat,0.504598,0.609381,1.000000,0.650995
소득세,0.428571,0.505217,0.733333,0.520651
양도소득세,0.227778,0.601601,1.000000,0.677477


In [ ]:
eval2 = evaluation('v2')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.744444
answer_relevancy     0.707000
context_recall       1.000000
context_precision    0.875164
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), 2. 적...",공동사업을 경영할 것을 약정하는 계약에 의한 토지 등의 현물출자는 양도세 과세 대상...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.823214,1.0,0.876190,2,3.699405,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제53조(생계를 같이 하는 부양가족의 범위와 그 판정시기), 제167조의10(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 생계를 같이 하는 부양가족의...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.0,0.971235,1.0,1.000000,2,3.971235,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[1. 납부할 양도소득세 계산식\n \n 거주주택 양도 당시 해당 임대주택 또는 어...,두 주택의 양도소득세는 각각의 양도소득에 대해 별도로 계산하여 높은 세율을 적용한 ...,1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,1.0,0.613975,1.0,1.000000,2,3.613975,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"증여자가 사망한 후 배우자가 양도하는 경우, 증여세 비과세 여부는 해당 증여가 상속...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.0,0.918737,1.0,0.638889,2,2.557626,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[가. [양도가액-(기존건물과 그 부수토지의 평가액-지급받은 청산금)-법 제97조 ...,국민주택채권 및 토지개발채권을 만기 전에 양도하여 발생하는 매각차손은 필요경비로 산...,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,1.0,0.646136,1.0,0.810000,2,3.456136,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.655000,0.523787,0.988889,0.938458
vat,0.562778,0.655901,0.966667,0.834712
소득세,0.516667,0.487365,0.788889,0.680380
양도소득세,0.744444,0.707000,1.000000,0.875164


In [ ]:
eval3 = evaluation('v3')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.721429
answer_relevancy     0.642470
context_recall       1.000000
context_precision    0.811778
dtype: float64
best performance: 1번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
0,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), ① 내국...","공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우, 양도소득...","현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",0.6,0.378530,1.0,0.976190,1,2.954720,양도소득세
1,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장...,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 생계를 같이 하는 부양가족의...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.0,0.972690,1.0,1.000000,1,3.972690,양도소득세
2,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[① 거주자의 양도소득에 대한 과세표준(이하 ""양도소득과세표준""이라 한다)은 종합소...","주택 2채를 동시에 양도할 경우, 각각의 양도소득세는 별도로 계산되어 큰 세액이 적...",1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,1.0,0.792055,1.0,1.000000,1,3.792055,양도소득세
3,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 1. 거주자가 그 배우자 또는 직계존비속으...","배우자가 사망 후 양도하는 경우, 증여세 과세가액에 산입되지 않은 경우를 제외하고는...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.5,0.722220,1.0,0.642857,1,2.865077,양도소득세
4,국민주택채권 등 매각차손 필요경비 산입 여부는?,[2. 법 제94조제1항제1호의 자산을 취득함에 있어서 법령등의 규정에 따라 매입한...,"국민주택채권 등 매각차손은 양도소득의 필요경비로 산입할 수 있으며, 이는 법령에 따...",소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,1.0,0.550078,1.0,0.892857,1,3.442935,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.472222,0.518062,0.933333,0.923283
vat,0.419444,0.707612,1.000000,0.810731
소득세,0.555556,0.444945,0.733333,0.728858
양도소득세,0.721429,0.642470,1.000000,0.811778


## testset 수정 후 실험 결과

In [ ]:
eval4 = evaluation('v4')

/tmp/ipython-input-2248849936.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-2248849936.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.504444
answer_relevancy     0.089769
context_recall       1.000000
context_precision    0.773333
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), ① 내국...",제공된 자료에서 관련 내용을 찾을 수 없습니다,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",0.5,0.000000,1.0,1.000000,2,2.500000,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장...,제공된 자료에서 관련 내용을 찾을 수 없습니다,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.0,0.000000,1.0,1.000000,2,2.000000,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 ...,"주택 2채를 동시에 양도하는 경우, 각각의 양도소득에 대해 별도로 과세표준을 계산한...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,1.0,0.814962,1.0,1.000000,2,3.814962,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,추가 확인 필요. 문맥에서는 증여자가 사망한 후 배우자가 양도하는 경우의 비과세 여...,소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.0,0.000000,1.0,0.638889,2,1.638889,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[3. 거주자가 해당 자산에 대하여 납부하였거나 납부할 증여세 상당액이 있는 경우 ...,제공된 자료에서 국민주택채권 등 매각차손이 필요경비에 산입된다는 명시적 근거는 없습...,"토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",1.0,0.000000,1.0,0.638889,2,2.638889,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.331032,0.108024,0.935714,0.774259
vat,0.492778,0.042767,0.983333,0.796620
소득세,0.422222,0.160921,0.866667,0.661157
양도소득세,0.504444,0.089769,1.000000,0.773333


In [ ]:
eval3 = evaluation('v3')

법인세법: 1번째 evaluation


/tmp/ipython-input-2248849936.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-2248849936.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.695556
answer_relevancy     0.555403
context_recall       0.933333
context_precision    0.830540
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), ① 내국...",제공된 자료에서 관련 내용을 찾을 수 없습니다.,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.000000,1.0,1.000000,2,3.000000,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장...,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 생계를 같이 하는 부양가족의...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.0,0.972800,1.0,0.948413,2,3.921213,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 ...,"두 주택의 양도소득세는 각각의 양도소득 산출세액을 계산한 후, 그 중 큰 금액을 과...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,1.0,0.603258,1.0,1.000000,2,3.603258,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 1. 거주자가 그 배우자 또는 직계존비속으...","증여자가 사망한 후 배우자가 양도하는 경우, 상속세 과세가액에 산입되지 아니한 출연...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.6,0.816609,1.0,0.684524,2,3.101132,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,"[제59조(유형자산의 취득에 소요된 보험차익 상당액의 필요경비계산), 제31조(보험...",국민주택채권 등 매각차손은 필요경비에 산입되지 않습니다.,"토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.0,0.678098,1.0,0.873413,2,2.551511,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.519444,0.489415,0.950000,0.686746
vat,0.422414,0.732954,0.948276,0.833430
소득세,0.644499,0.457223,0.827586,0.624777
양도소득세,0.695556,0.555403,0.933333,0.830540


In [ ]:
eval2 = evaluation('v2')

법인세법: 1번째 evaluation


/tmp/ipython-input-2248849936.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-2248849936.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.381111
answer_relevancy     0.602149
context_recall       0.966667
context_precision    0.759762
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), ① 내국...",공동사업을 경영할 것을 약정하는 계약에 의한 토지 등의 현물출자는 양도소득세 과세 ...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.838801,1.0,0.948413,2,3.787213,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[①「소득세법」(이하 ""법""이라 한다) 제1조의2에 따른 주소는 국내에서 생계를 같...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 생계를 같이 하는 부양가족으...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.0,0.972800,1.0,1.000000,2,3.972800,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 ...,"두 주택을 같은 날 양도하는 경우, 거주자가 선택하는 순서에 따라 양도한 것으로 간...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,1.0,0.382741,1.0,0.948413,2,3.331154,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,제공된 자료에서 관련 내용을 찾을 수 없습니다.,소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.0,0.000000,1.0,0.226190,2,1.226190,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[2. 법 제94조제1항제1호의 자산을 취득함에 있어서 법령등의 규정에 따라 매입한...,국민주택채권 등 매각차손은 필요경비로 산입되지 않습니다.,"토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.0,0.834380,1.0,0.667857,2,2.502238,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.627778,0.465698,0.966667,0.719894
vat,0.475000,0.627051,0.983333,0.843567
소득세,0.541667,0.410219,0.688889,0.754570
양도소득세,0.381111,0.602149,0.966667,0.759762


In [ ]:
eval1 = evaluation('v1')

법인세법: 1번째 evaluation


/tmp/ipython-input-2248849936.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-2248849936.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.305556
answer_relevancy     0.588231
context_recall       0.933333
context_precision    0.638488
dtype: float64
best performance: 1번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
0,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,공동사업을 경영할 것을 약정하는 계약에 의한 토지 등의 현물출자는 양도세 과세 대상...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.00,0.773948,1.0,0.366667,1,3.140615,양도소득세
1,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 세법상 가족으로서 배우자와 ...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.00,0.972800,1.0,0.982143,1,2.954943,양도소득세
2,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도한 경우, 각각의 양도소득에 대해 별도로 양도소득세가 과세되...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,0.00,0.757960,1.0,0.614064,1,2.372025,양도소득세
3,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","배우자가 사망 후 양도하는 경우, 해당 양도는 증여로 간주되지 않으므로 비과세 여부...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.25,0.000000,1.0,0.666098,1,1.916098,양도소득세
4,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,제공된 자료에서 국민주택채권 등 매각차손 필요경비 산입 여부에 관한 구체적인 내용은...,"토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",1.00,0.000000,1.0,0.544414,1,2.544414,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.494444,0.479832,0.916667,0.551755
vat,0.482222,0.542938,0.983333,0.583799
소득세,0.463889,0.471377,0.833333,0.522436
양도소득세,0.305556,0.588231,0.933333,0.638488


In [ ]:
eval0 = evaluation('v0')

법인세법: 1번째 evaluation


/tmp/ipython-input-2248849936.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-2248849936.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[10]: TimeoutError()


법인세법: 2번째 evaluation


...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.404444
answer_relevancy     0.628165
context_recall       1.000000
context_precision    0.768328
dtype: float64
best performance: 3번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
10,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,공동사업을 경영할 것을 약정하는 계약에 의한 토지 등의 현물출자는 양도세 과세 대상...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.852606,1.0,0.383333,3,3.235939,양도소득세
11,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 제83조의 규정에 따른 1세...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.0,0.972800,1.0,1.000000,3,2.972800,양도소득세
12,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도하는 경우, 제55조제1항에 따른 세율에 100분의 20 또...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,1.0,0.737615,1.0,0.915079,3,3.652695,양도소득세
13,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","증여자가 사망한 후 배우자가 양도하는 경우, 비과세 여부는 해당 양도에 대한 구체적...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.5,0.000000,1.0,0.854167,3,2.354167,양도소득세
14,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,국민주택채권 등 매각차손은 필요경비에 산입되지 않습니다.,"토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.0,0.838411,1.0,0.734524,3,2.572935,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.469444,0.571122,0.931034,0.611556
vat,0.630556,0.522546,1.000000,0.659163
소득세,0.500000,0.485704,0.684921,0.612533
양도소득세,0.404444,0.628165,1.000000,0.768328


## initial_k, final_k, top_k 조정

In [19]:
nest_asyncio.apply()

logging.getLogger("ragas").setLevel(logging.ERROR)
logging.getLogger("openai").setLevel(logging.ERROR)

def evaluation_tuned(version:str, n_tries: int = 3, initial_k=15, final_k=20, top_k=7):
  file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json']
  category = ['법인세법', 'vat', '소득세', '양도소득세']

  final_summary = []
  best_perf = []

  llm_ragas = LangchainLLMWrapper(llm)
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

  run_config = RunConfig(max_retries=3, timeout=180)

  for path, law_name in zip(file_path, category):

    try:
      with open(path, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

      try_results = []

      for i in range(n_tries):
        print(f"{law_name}: {i+1}번째 evaluation")

        answers = []
        contexts = []
        questions = []
        ground_truths = []

        for item in test_data:
          query = item['query']
          try:
            if version == 'v4':
              pipeline = Pipeline4(query, llm, vectorstore)
            elif version == 'v3':
              pipeline = Pipeline(query, llm, vectorstore, initial_k, final_k, top_k)
            elif version == 'v3_advanced':
              pipeline = AdvancedPipeline(query, llm, vectorstore, initial_k, final_k, top_k)
            elif version == 'v2':
              pipeline = MultiQueryPipeline(query, llm, vectorstore)
            elif version == 'v1':
              pipeline = ParentPipeline(query, vectorstore, llm)
            elif version == 'v0':
              pipeline = BasicPipeline(query, llm, vectorstore)
            else:
              raise ValueError("v0, v1, v2, v3, v4 중 선택하세요.")

            answer, _, retrieved_docs = pipeline.generate_answer()
            answers.append(answer)
            contexts.append([doc.page_content for doc in retrieved_docs])
            questions.append(query)
            ground_truths.append(item['answer'])

          except Exception as e:
            print(f"평가 중 에러 발생: {e}")
            continue

        data = {
          "user_input": questions,
          "response": answers,
          "retrieved_contexts": contexts,
          "reference": ground_truths
        }

        dataset = Dataset.from_dict(data)

        result = evaluate(
            dataset=dataset,
            metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
            llm=llm_ragas,
            embeddings=embedding_ragas
        )

        df_try = result.to_pandas()
        df_try['try_no'] = i + 1
        try_results.append(df_try)

      if try_results:
        all_tries_df = pd.concat(try_results, ignore_index=True)
        cols = ['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']

        avg = all_tries_df[cols].mean()
        print(f"{law_name} {n_tries}회 평균")
        print(avg)

        all_tries_df['sum'] = all_tries_df[cols].sum(axis=1)
        try_scores = all_tries_df.groupby('try_no')['sum'].mean()
        best_try_idx = int(try_scores.idxmax())
        best_try = all_tries_df[all_tries_df['try_no'].values == best_try_idx].copy()
        best_try['law_name'] = law_name

        print(f"best performance: {best_try_idx}번째")
        display(best_try)
        best_perf.append(best_try)

        summary = avg.to_frame().T
        summary['law_name'] = law_name
        final_summary.append(summary)

    except Exception as e:
      print(f"{law_name} 평가 중 오류 발생: {e}")


  if final_summary:
      final_report = pd.concat(final_summary, ignore_index=True)
      print("-"*50)
      print("최종 평가 결과 요약")
      display(final_report.set_index('law_name'))
      return final_report

  return None

In [14]:
from V3_advanced import AdvancedPipeline

In [16]:
initial_k = 15
final_k = 10
top_k = 5

eval3_tuned = evaluation_tuned('v3_advanced', initial_k=initial_k, final_k=final_k, top_k=top_k)

/tmp/ipython-input-1548958818.py:13: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1548958818.py:14: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.737037
answer_relevancy     0.419810
context_recall       1.000000
context_precision    0.854167
dtype: float64
best performance: 3번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
10,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[① 내국법인(이하 이 조에서 ""출자법인""이라 한다)이 다음 각 호의 요건을 갖춘 ...",결론(조건부): 공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.000000,1.0,1.00,3,3.000000,양도소득세
11,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장...,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 법 제4조에 따른 1세대의 ...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.5,0.972800,1.0,1.00,3,3.472800,양도소득세
12,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 ...,"양도소득세는 각각의 주택 양도에 대해 별도로 계산하며, 두 주택의 양도소득세를 각각...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,1.0,0.506560,1.0,0.95,3,3.456560,양도소득세
13,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 거주자의 배우자로서 해당 과세기간의 ...","결론(조건부): 배우자가 사망 후 양도하는 경우, 증여세 과세 여부는 해당 양도 재...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,1.0,0.742142,1.0,0.70,3,3.442142,양도소득세
14,국민주택채권 등 매각차손 필요경비 산입 여부는?,[3. 거주자가 해당 자산에 대하여 납부하였거나 납부할 증여세 상당액이 있는 경우 ...,"결론(조건부): 국민주택채권 등 매각차손은 필요경비에 산입할 수 있으며, 다만 관련...","토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.5,0.907561,1.0,1.00,3,3.407561,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.602222,0.404037,0.933333,0.810880
vat,0.662222,0.349108,1.000000,0.862130
소득세,0.685000,0.461277,0.828571,0.688519
양도소득세,0.737037,0.419810,1.000000,0.854167


In [17]:
initial_k = 25
final_k = 15
top_k = 7

eval3_tuned = evaluation_tuned('v3_advanced', initial_k=initial_k, final_k=final_k, top_k=top_k)

/tmp/ipython-input-1548958818.py:13: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1548958818.py:14: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.712222
answer_relevancy     0.299305
context_recall       0.933333
context_precision    0.882807
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[① 내국법인(이하 이 조에서 ""출자법인""이라 한다)이 다음 각 호의 요건을 갖춘 ...",결론(조건부): 공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.000000,0.349599,1.0,1.000000,2,3.349599,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[제167조의10(양도소득세가 중과되는 1세대 2주택에 해당하는 주택의 범위) 제1...,결론(조건부): 1세대 판단 시 가족의 범위는 소득세법 시행령 제167조의10과 제...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.000000,0.623690,1.0,1.000000,2,3.623690,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 ...,"결론(확정): 주택 2채를 동시에 양도하는 경우, 각각의 양도소득금액에 대해 양도소...",1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,0.800000,0.333812,1.0,0.876667,2,3.010478,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"결론(조건부): 증여자가 사망한 후 배우자가 양도하는 경우, 증여세 비과세 여부는 ...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,1.000000,0.000000,1.0,0.500000,2,2.500000,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,"[제59조(유형자산의 취득에 소요된 보험차익 상당액의 필요경비계산), 제31조(보험...","국민주택채권 등 매각차손은 필요경비에 산입하지 않으며, 관련 규정이 명시되어 있지 ...","토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.666667,0.000000,1.0,0.852857,2,2.519524,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.571825,0.421218,0.966667,0.774709
vat,0.708889,0.422549,0.988889,0.807286
소득세,0.611667,0.490062,0.750000,0.647632
양도소득세,0.712222,0.299305,0.933333,0.882807


In [18]:
initial_k = 25
final_k = 15
top_k = 4

eval3_tuned = evaluation_tuned('v3_advanced', initial_k=initial_k, final_k=final_k, top_k=top_k)

/tmp/ipython-input-1548958818.py:13: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1548958818.py:14: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.727778
answer_relevancy     0.488796
context_recall       1.000000
context_precision    0.894444
dtype: float64
best performance: 3번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
10,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), 2. 적...",결론(조건부): 공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.000000,0.805445,1.0,1.000000,3,3.805445,양도소득세
11,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,[3. 법 제4조에 따른 소득 중 재정경제부령으로 정하는 소득이 「국민기초생활 보장...,결론(조건부): 양도소득세에서 1세대 판단 시 가족의 범위는 법 제4조에 따른 1세...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.000000,0.946941,1.0,1.000000,3,3.946941,양도소득세
12,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[⑨법 제89조제1항제3호의 규정을 적용함에 있어서 2개 이상의 주택을 같은 날에 ...,결론(확정): 주택 2채를 같은 날 양도하는 경우에는 양도하는 순서에 따라 각각 양...,1세대 1주택 비과세 요건을 충족한 2개 이상의 주택을 같은 날 양도할 경우 그 중...,0.666667,0.837643,1.0,0.916667,3,3.420977,양도소득세
13,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 1. 거주자가 그 배우자 또는 직계존비속으...","결론(조건부): 배우자가 사망한 후 양도하는 경우, 증여세 과세가액에 산입되지 않은...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,1.000000,0.074152,1.0,0.833333,3,2.907485,양도소득세
14,국민주택채권 등 매각차손 필요경비 산입 여부는?,"[제59조(유형자산의 취득에 소요된 보험차익 상당액의 필요경비계산), 제31조(보험...","국민주택채권 등 매각차손은 필요경비에 산입되지 않으며, 관련 규정에서 명시적으로 제...","토지, 건물을 취득함에 있어서 법령 등의 규정에 따라 매입한 국민주택채권 및 토지개...",0.000000,0.366016,1.0,0.916667,3,2.282683,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.682222,0.351774,0.944444,0.823148
vat,0.707778,0.519856,0.983333,0.924074
소득세,0.651111,0.506855,0.866667,0.595370
양도소득세,0.727778,0.488796,1.000000,0.894444
